In [ ]:
from data_loading.data_loader import load_data

In [2]:
data = load_data()

/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
import torch
import numpy as np

In [4]:
np.random.shuffle(data)

In [5]:
X = []
y = []
for d in data:
    X.append(d[0])
    y.append(d[1])
VAL_SPLIT = 0.25
VAL_SPLIT = len(X) * VAL_SPLIT
VAL_SPLIT = int(VAL_SPLIT)
X_train = X[:-VAL_SPLIT]
y_train = y[:-VAL_SPLIT]
X_test = X[-VAL_SPLIT:]
y_test = y[-VAL_SPLIT:]
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
X_train = torch.from_numpy(np.array(X_train))
y_train = torch.from_numpy(np.array(y_train))
X_test = torch.from_numpy(np.array(X_test))
y_test = torch.from_numpy(np.array(y_test))
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

6276
6276
2091
2091
6276
6276
2091
2091


## Modelling

In [6]:
from torchvision import models

In [7]:
device = torch.device('cuda')

In [8]:
import torch.nn as nn

In [ ]:
# model = models.resnet18(pretrained=True).to(device)
# inf = model.fc.in_features
# model.fc = nn.Linear(inf,2)
from models.baseline_model import BaseLine_Model
model = BaseLine_Model().to(device)

In [ ]:
model = model.to(device)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [ ]:
BATCH_SIZE = 32

In [ ]:
EPOCHS = 100

In [ ]:
import wandb

In [ ]:
PROJECT_NAME = 'Face-Mask-Detection'

In [ ]:
from tqdm import tqdm

In [ ]:
wandb.init(project=PROJECT_NAME,name='test')
for _ in tqdm(range(EPOCHS),leave=False):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        preds = model(X_batch)
        preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'loss':loss.item()})
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)


  0%|          | 0/100 [00:00<?, ?it/s]

tensor([[-0.0445, -0.0434],
        [-0.0427, -0.0414],
        [-0.0423, -0.0471],
        [-0.0468, -0.0444],
        [-0.0454, -0.0480],
        [-0.0467, -0.0437],
        [-0.0514, -0.0486],
        [-0.0445, -0.0509],
        [-0.0440, -0.0447],
        [-0.0434, -0.0485],
        [-0.0387, -0.0464],
        [-0.0428, -0.0480],
        [-0.0451, -0.0455],
        [-0.0417, -0.0438],
        [-0.0420, -0.0434],
        [-0.0458, -0.0410],
        [-0.0458, -0.0453],
        [-0.0458, -0.0454],
        [-0.0438, -0.0399],
        [-0.0467, -0.0481],
        [-0.0435, -0.0434],
        [-0.0433, -0.0451],
        [-0.0468, -0.0447],
        [-0.0404, -0.0473],
        [-0.0440, -0.0449],
        [-0.0462, -0.0470],
        [-0.0476, -0.0499],
        [-0.0421, -0.0401],
        [-0.0448, -0.0456],
        [-0.0470, -0.0463],
        [-0.0451, -0.0491],
        [-0.0447, -0.0486]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[-0.0264, -0.0664],
        [-0.0284, -0.0620],
     

In [ ]:
for index in range(len(preds)):
    print(torch.argmax(torch.round(preds)[index]))
    print(y_batch[index])
    print('\n')